In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thesis-chess-dataset/fourthSorted.pgn


In [2]:
import os

# Splits the PGN file into parts, each containing up to 'split_size' games from a specific year, and saves them to the output folder
def split_and_filter_games_by_year(pgn_file_path, year, split_size, output_folder):
    
    os.makedirs(output_folder, exist_ok=True)
    
    current_file_number = 1
    game_count = 0
    current_file_path = os.path.join(output_folder, f"games_2021_part_{current_file_number}.pgn")
    current_file = open(current_file_path, "w")
    
    with open(pgn_file_path, "r") as pgn:
        game_lines = []  # Hold the lines of a game
        for line in pgn:
            if line.startswith('[Event '):
                # Process the previous game
                if game_lines:
                    date_line = next((l for l in game_lines if l.startswith('[Date ')), '')
                    game_year = date_line.split('"')[1].split('.')[0]  # Extract the year part
                    if game_year == year:
                        game_count += 1
                        if game_count > split_size:
                            # Close current file and open a new one
                            current_file.close()
                            current_file_number += 1
                            current_file_path = os.path.join(output_folder, f"games_2021_part_{current_file_number}.pgn")
                            current_file = open(current_file_path, "w")
                            game_count = 1  # Reset the count for the new file
                        current_file.writelines(game_lines)
                game_lines = [line]  # Start a new game
            else:
                game_lines.append(line)
        
        # Check for the last game in the file
        if game_lines:
            date_line = next((l for l in game_lines if l.startswith('[Date ')), '')
            game_year = date_line.split('"')[1].split('.')[0]
            if game_year == year:
                current_file.writelines(game_lines)
                
    current_file.close()  # Ensure the last file is closed after the loop finishes

# Path to the directory containing the PGN file
pgn_file_path = "/kaggle/input/thesis-chess-dataset/fourthSorted.pgn"
# Selection of filter for year
year = "2021"
# File size for split by no. of games
split_size = 10000

output_folder = "pgn_2021_data"

# Call the function
split_and_filter_games_by_year(pgn_file_path, year, split_size, output_folder)

In [3]:
pip install converter pgn2data

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.3 MB/s eta 0:00:00
  Created wheel for converter: filename=converter-1.0.0-py3-none-any.whl size=1751 sha256=27ea29e5fd1750ede8ffb7ff8ea7e4ed3c884d451ca804b9d270f95cc13ff385
  Stored in directory: /root/.cache/pip/wheels/fd/74/44/8a22cacdcff0e3a951e569210219b458168eba1e34f03e24f3
Successfully built converter
Note: you may need to restart the kernel to use updated packages.


In [4]:
from converter.pgn_data import PGNData
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

# function to process pgn to csv with games info and moves csv files
def process_pgn_file(pgn_file_path):

    try:
        pgn_data = PGNData(pgn_file_path)
        result = pgn_data.export()
        result.print_summary()  # Print summary of the export
        return f"Processing {pgn_file_path} completed successfully."
    except Exception as e:
        return f"Error processing {pgn_file_path}: {e}"

def process_pgn_files_in_parallel(files):
    """
    Process a list of PGN files in parallel.
    """
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = {executor.submit(process_pgn_file, file): file for file in files}
        for future in as_completed(futures):
            print(future.result())

# Path to the directory containing the PGN files
pgn_directory = "/kaggle/working/pgn_2021_data"

# Get all PGN files in the directory
pgn_files = [os.path.join(pgn_directory, f) for f in os.listdir(pgn_directory) if f.endswith('.pgn')]

# Process in batches of 12
batch_size = 12
for i in range(0, len(pgn_files), batch_size):
    batch_files = pgn_files[i:i+batch_size]
    process_pgn_files_in_parallel(batch_files)

is complete: True
games file: games_2021_part_8_game_info.csv | size: 2631302
moves file: games_2021_part_8_moves.csv | size: 354112694
Processing /kaggle/working/pgn_2021_data/games_2021_part_8.pgn completed successfully.
is complete: True
games file: games_2021_part_28_game_info.csv | size: 2539615
moves file: games_2021_part_28_moves.csv | size: 365621008
Processing /kaggle/working/pgn_2021_data/games_2021_part_28.pgn completed successfully.
is complete: True
games file: games_2021_part_26_game_info.csv | size: 2562070
moves file: games_2021_part_26_moves.csv | size: 369277492
Processing /kaggle/working/pgn_2021_data/games_2021_part_26.pgn completed successfully.
is complete: True
games file: games_2021_part_5_game_info.csv | size: 2700283
moves file: games_2021_part_5_moves.csv | size: 378164544
Processing /kaggle/working/pgn_2021_data/games_2021_part_5.pgn completed successfully.
is complete: True
games file: games_2021_part_18_game_info.csv | size: 2610735
moves file: games_2021_